<a href="https://colab.research.google.com/github/Johnfratelli/caminhos-da-estatistica/blob/main/caminhos_da_estatistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import re
import unicodedata
import itertools
from datasets import Dataset

# Library for file manipulation
import pandas as pd
import numpy as np
import pandas

# Data visualization
import seaborn as sns
import matplotlib.pylab as pl
import matplotlib as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot as plt

# ML NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [59]:
review = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
review = review["full"][0]
print(review)

produto = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full", trust_remote_code=True)
print(produto[0])

{'rating': 5.0, 'title': 'Such a lovely scent but not overpowering.', 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!", 'images': [], 'asin': 'B00YQ6X8EO', 'parent_asin': 'B00YQ6X8EO', 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ', 'timestamp': 1588687728923, 'helpful_vote': 0, 'verified_purchase': True}
{'main_category': 'All Beauty', 'title': 'Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)', 'average_rating': 4.8, 'rating_number': 10, 'features': [], 'description': [], 'price': 'None', 'images': {'hi_res': [None, 'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg', 'https://m.media-amazon.com/images/I/41w2yznfuZL.jpg'], 'thumb': ['https://m.media-ama

In [60]:
#Demora uns 2 min para conferter od dataset para o pandas.
import pandas as pd

review_df = pd.DataFrame(review)

produto_df = pd.DataFrame(produto)

In [34]:
review_df.head()

,full
0,"{'rating': 5.0, 'title': 'Such a lovely scent ..."
1,"{'rating': 4.0, 'title': 'Works great but smel..."
2,"{'rating': 5.0, 'title': 'Yes!', 'text': 'Smel..."
3,"{'rating': 1.0, 'title': 'Synthetic feeling', ..."
4,"{'rating': 5.0, 'title': 'A+', 'text': 'Love i..."


No charts were generated by quickchart


In [47]:
produto_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [36]:
review_df.tail()
produto_df.tail()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
112585,All Beauty,"TOPREETY 24""120gr 3/4 Full Head clip in hair e...",5.0,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",TOPREETY,[],"{""Is Discontinued By Manufacturer"": ""No"", ""Pac...",B077D2Z5RF,None,None,None
112586,All Beauty,"Pets Playmate Pet Grooming Glove,Gentle Deshed...",5.0,1,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Pets Playmate,[],"{""Is Discontinued By Manufacturer"": ""No"", ""Pac...",B07DLRYKQZ,None,None,None
112587,All Beauty,[10Pack] Makeup Brushes Set Cosmetics Tools Ki...,4.3,5,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",RainMakers,[],"{""Brand"": ""RainMakers"", ""Recommended Uses For ...",B07HNP2NTF,None,None,None
112588,All Beauty,Xcoser Pretty Party Anna Wig Hair Tails Hair S...,4.0,9,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Xcoser,[],"{""Is Discontinued By Manufacturer"": ""No"", ""Pac...",B00U3OB8PY,None,None,None
112589,All Beauty,"DVIO Men's Voyage Perfume, Spicy woody fragran...",1.8,2,"[Features: Spicy woody notes, pure sandalwood ...",[],27.99,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",DVIO,[],"{""UPC"": ""736238729337"", ""Manufacturer"": ""DVIO""}",B09B9TCYWG,None,None,None


In [48]:
review_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701528 entries, 0 to 701527
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   full    701528 non-null  object
dtypes: object(1)
memory usage: 5.4+ MB


In [39]:
produto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112590 entries, 0 to 112589
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   main_category    112590 non-null  object 
 1   title            112590 non-null  object 
 2   average_rating   112590 non-null  float64
 3   rating_number    112590 non-null  int64  
 4   features         112590 non-null  object 
 5   description      112590 non-null  object 
 6   price            112590 non-null  object 
 7   images           112590 non-null  object 
 8   videos           112590 non-null  object 
 9   store            101259 non-null  object 
 10  categories       112590 non-null  object 
 11  details          112590 non-null  object 
 12  parent_asin      112590 non-null  object 
 13  bought_together  0 non-null       object 
 14  subtitle         0 non-null       object 
 15  author           0 non-null       object 
dtypes: float64(1), int64(1), object(14)
me

In [44]:
review_df.shape




(701528, 1)

In [45]:
produto_df.shape

(112590, 16)

In [50]:
# prompt: faça a união das variaveis review_df e produto_df unindo pelo parent_asin e nomei a nova variavel como data

data = pd.merge(review_df, produto_df, on='parent_asin', how='inner')

KeyError: 'parent_asin'

In [56]:
print(review_df)

                                                     full
0       {'rating': 5.0, 'title': 'Such a lovely scent ...
1       {'rating': 4.0, 'title': 'Works great but smel...
2       {'rating': 5.0, 'title': 'Yes!', 'text': 'Smel...
3       {'rating': 1.0, 'title': 'Synthetic feeling', ...
4       {'rating': 5.0, 'title': 'A+', 'text': 'Love i...
...                                                   ...
701523  {'rating': 4.0, 'title': 'Four Stars', 'text':...
701524  {'rating': 1.0, 'title': 'Pretty', 'text': 'Di...
701525  {'rating': 5.0, 'title': 'Great sunless tanner...
701526  {'rating': 5.0, 'title': 'The Crown on top is ...
701527  {'rating': 4.0, 'title': 'Good Shampoo/Conditi...

[701528 rows x 1 columns]
